In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import mean, abs
from collections import Counter
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType
import pandas as pd

In [2]:
class PSTool:
    def pyspark_session(self, host_location):
        """
        Creates and returns spark session object
        """
        sc = SparkContext(host_location)  # Create spark context
        spark = SparkSession(sc)  # Create session
        return spark

    def file_loader(self, path, delim, spark_obj):
        data = spark_obj.read.options(delimiter=delim).csv(path)  # load file
        data = data
        return data


if __name__ == "__main__":
    # Instanciate object
    pstool = PSTool()
    # start session
    spk = pstool.pyspark_session('local')
    # load data
    path = 'all_bacilli_subset.tsv'
    df=pstool.file_loader(path, '\t', spk)

The TSV format presents the match data in columns as follows:

    1. Protein accession (e.g. P51587)
    2. Sequence MD5 digest (e.g. 14086411a2cdf1c4cba63020e1622579)
    3. Sequence length (e.g. 3418)
    4. Analysis (e.g. Pfam / PRINTS / Gene3D)
    5. Signature accession (e.g. PF09103 / G3DSA:2.40.50.140)
    6. Signature description (e.g. BRCA2 repeat profile)
    7. Start location
    8. Stop location
    9. Score - is the e-value (or score) of the match reported by member database method (e.g. 3.1E-52)
    10. Status - is the status of the match (T: true)
    11. Date - is the date of the run
    12. InterPro annotations - accession (e.g. IPR002093)
    13. InterPro annotations - description (e.g. BRCA2 repeat)
    14. (GO annotations (e.g. GO:0005515) - optional column; only displayed if –goterms option is switched on)
    15. (Pathways annotations (e.g. REACT_71)-optional column; only displayed if –pathways option is switched on)


In [3]:
df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string]

# Question 1
How many distinct protein annotations are found in the dataset? I.e. how many distinc InterPRO numbers are there?

In [4]:
# df.select("_c11").distinct().count().explain()
Q1_exp = df.select("_c11").filter(df._c11 != '-').distinct()._jdf.queryExecution().simpleString().replace('\n', '')
Q1_answer = df.select("_c11").filter(df._c11 != '-').distinct().count()
Q1_answer

60

# Question 2
How many annotations does a protein have on average?

In [5]:
Q2_exp = df.select("_c11").filter(df._c11 != '-').distinct()._jdf.queryExecution().simpleString().replace('\n', '')
# Q2_2_exp = df.select("_c11").filter(df._c11 != '-').explain()
Q2_answer = df.select("_c11").filter(df._c11 != '-').count()/df.select("_c11").filter(df._c11 != '-').distinct().count()
Q2_answer

1.8

# Question 3
What is the most common GO Term found?

In [6]:
from pyspark.sql.functions import *
Q3_exp = df.withColumn("_c13",explode(split(col("_c13"),'\\|'))).select("_c13").filter(df._c13 != '-').agg({'_c13': 'max'})._jdf.queryExecution().simpleString().replace('\n', '')
Q3_answer = df.withColumn("_c13",explode(split(col("_c13"),'\\|'))).select("_c13").filter(df._c13 != '-').agg({'_c13': 'max'}).collect()[0][0]
Q3_answer

'GO:0071973'

# Question 4
What is the average size of an InterPRO feature found in the dataset?

In [7]:
Q4_exp = df.select(abs(df._c7 - df._c8)).agg({'abs((_c7 - _c8))': 'mean'})._jdf.queryExecution().simpleString().replace('\n', '')
Q4_answer = df.select(abs(df._c7 - df._c8)).agg({'abs((_c7 - _c8))': 'mean'}).collect()[0][0]
Q4_answer

275.50452025394884

# Question 5
What is the top 10 most common InterPRO features?

In [8]:
Q5_exp = df.filter(df._c11 != '-').groupBy('_c11').count()._jdf.queryExecution().simpleString().replace('\n', '')

df_g = df.filter(df._c11 != '-').groupBy('_c11').count()
Q5_answer = df_g.orderBy(df_g['count'].desc()).head(10)

Q5_answer = [Q5_answer[n].__getitem__('_c11') for n, i in enumerate(Q5_answer)]
Q5_answer

['IPR001697',
 'IPR005814',
 'IPR000456',
 'IPR001867',
 'IPR004358',
 'IPR000515',
 'IPR001789',
 'IPR036890',
 'IPR003594',
 'IPR015963']

# Question 6
If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then?

In [9]:
Q6_exp = df.withColumn('perc', (df._c7 - df._c8) / df._c2).sort('perc')._jdf.queryExecution().simpleString().replace('\n', '')
df_2 = df.withColumn('perc', (df._c7 - df._c8) / df._c2).sort('perc')
Q6_answer = df_2.filter(df._c11 != '-').filter(df_2.perc > 0.9).groupBy('_c11').count().sort('count', ascending = False).head(10)
Q6_answer = [Q6_answer[n].__getitem__('_c11') for n, i in enumerate(Q6_answer)]
Q6_answer

['IPR036890',
 'IPR005814',
 'IPR003594',
 'IPR004358',
 'IPR001867',
 'IPR000456',
 'IPR035906',
 'IPR007466',
 'IPR015963',
 'IPR036393']

# Question 7
If you look at those features which also have textual annotation, what is the top 10 most common word found in that annotation?
https://stackoverflow.com/questions/65808504/spark-find-most-common-words-in-dataset

In [10]:
Q7_exp = df.filter(df._c12 != '-').select('_c12').withColumn("_c12",explode(split(col("_c12"),' '))).groupBy('_c12').count().sort('count', ascending = False)._jdf.queryExecution().simpleString().replace('\n', '')
Q7_answer = df.filter(df._c12 != '-').select('_c12').withColumn("_c12",explode(split(col("_c12"),' '))).groupBy('_c12').count().sort('count', ascending = False).head(10)
Q7_answer = [Q7_answer[n].__getitem__('_c12') for n, i in enumerate(Q7_answer)]
Q7_answer

['domain',
 'superfamily',
 'Pyruvate',
 'protein',
 'kinase',
 'kinase,',
 'C-terminal',
 'L17',
 'Histidine',
 'transduction']

# Question 8
And the top 10 least common?

In [11]:
Q8_exp = df.filter(df._c12 != '-').select('_c12').withColumn("_c12",explode(split(col("_c12"),' '))).groupBy('_c12').count().sort('count', ascending = True)._jdf.queryExecution().simpleString().replace('\n', '')
Q8_answer = df.filter(df._c12 != '-').select('_c12').withColumn("_c12",explode(split(col("_c12"),' '))).groupBy('_c12').count().sort('count', ascending = True).head(10)
Q8_answer = [Q8_answer[n].__getitem__('_c12') for n, i in enumerate(Q8_answer)]
Q8_answer

['GAF',
 'TipAS',
 'biosynthesis,',
 'isomerase',
 'mobile',
 'DUF1657',
 'major',
 'component',
 'Bacteriocin',
 'ArtM/GltK/GlnP/TcyL/YhdX-like']

# Question 9
Combining your answers for Q6 and Q7, what are the 10 most commons words found for the largest InterPRO features?

In [12]:
Q9_exp = df.where(f.col('_c11').isin(Q6_answer)).filter(df._c12 != '-').select('_c12').withColumn("_c12",explode(split(col("_c12"),' '))).groupBy('_c12').count().sort('count', ascending = False)._jdf.queryExecution().simpleString().replace('\n', '')
df3 = df.where(f.col('_c11').isin(Q6_answer)).filter(df._c12 != '-').select('_c12').withColumn("_c12",explode(split(col("_c12"),' '))).groupBy('_c12').count().sort('count', ascending = False).head(10)
Q9_answer = [df3[n].__getitem__('_c12') for n, i in enumerate(df3)]
Q9_answer

['superfamily',
 'kinase/HSP90-like',
 'ATPase',
 'Histidine',
 'Aminotransferase',
 'class-III',
 'L17',
 'protein',
 'Ribosomal',
 'kinase-related']

# Question 10
What is the coefficient of correlation ( 𝑅2 ) between the size of the protein and the number of features found?

In [13]:
Q10_exp = df.select(df._c0,df._c11,df._c2).filter(df._c11 != "-").groupby(df._c0,"_c2").count()._jdf.queryExecution().simpleString().replace('\n', '')
Q10 = df.select(df._c0,df._c11,df._c2).filter(df._c11 != "-").groupby(df._c0,"_c2").count()

Q10_answer = Q10.withColumn('_c2' , Q10['_c2'].cast(FloatType()))
Q10_answer = Q10_answer.corr('_c2', 'count')**2
Q10_answer

0.14455840022857916

## Writing to CSV

In [14]:
Questnum = list(range(1,11))
Answers = [Q1_answer, Q2_answer, Q3_answer, Q4_answer, Q5_answer, Q6_answer, Q7_answer, Q8_answer, Q9_answer, Q10_answer]
Explains = [Q1_exp, Q2_exp, Q3_exp, Q4_exp, Q5_exp, Q6_exp, Q7_exp, Q8_exp, Q9_exp, Q10_exp]

zipped = list(zip(Questnum, Answers, Explains))
data = pd.DataFrame(zipped, columns = ['Questnum', 'Answers', 'Explains'])
data

,Questnum,Answers,Explains
0,1,60,== Physical Plan ==AdaptiveSparkPlan isFinalPl...
1,2,1.8,== Physical Plan ==AdaptiveSparkPlan isFinalPl...
2,3,GO:0071973,== Physical Plan ==AdaptiveSparkPlan isFinalPl...
3,4,275.50452,== Physical Plan ==AdaptiveSparkPlan isFinalPl...
4,5,"[IPR001697, IPR005814, IPR000456, IPR001867, I...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
5,6,"[IPR036890, IPR005814, IPR003594, IPR004358, I...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
6,7,"[domain, superfamily, Pyruvate, protein, kinas...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
7,8,"[GAF, TipAS, biosynthesis,, isomerase, mobile,...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
8,9,"[superfamily, kinase/HSP90-like, ATPase, Histi...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
9,10,0.144558,== Physical Plan ==AdaptiveSparkPlan isFinalPl...


In [15]:
data.to_csv("assignment5.csv",index=False)

In [17]:
test = pd.read_csv('assignment5.csv')
test

,Questnum,Answers,Explains
0,1,60,== Physical Plan ==AdaptiveSparkPlan isFinalPl...
1,2,1.8,== Physical Plan ==AdaptiveSparkPlan isFinalPl...
2,3,GO:0071973,== Physical Plan ==AdaptiveSparkPlan isFinalPl...
3,4,275.50452025394884,== Physical Plan ==AdaptiveSparkPlan isFinalPl...
4,5,"['IPR001697', 'IPR005814', 'IPR000456', 'IPR00...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
5,6,"['IPR036890', 'IPR005814', 'IPR003594', 'IPR00...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
6,7,"['domain', 'superfamily', 'Pyruvate', 'protein...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
7,8,"['GAF', 'TipAS', 'biosynthesis,', 'isomerase',...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
8,9,"['superfamily', 'kinase/HSP90-like', 'ATPase',...",== Physical Plan ==AdaptiveSparkPlan isFinalPl...
9,10,0.14455840022857916,== Physical Plan ==AdaptiveSparkPlan isFinalPl...
